In [1]:
import pathlib
import os#, shutil
#import matplotlib.pyplot as plt
#import random
import json

from PIL import Image, ImageOps
from IPython.display import Image as DImage

# from keras import layers, models
# from keras.models import Sequential, load_model
# from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
# from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing import image

# from tensorflow.keras import optimizers
# from tensorflow.keras.utils import load_img, img_to_array, array_to_img

In [2]:
#Класс для обрезки изобрадения согласно формату 
class CutImageUtil(object):
    def __get_bounds(self,image, srs):
        x1, y1, w, h = srs
        width, height = image.size

        return x1 * width, y1 * height, (x1 + w) * width, (y1 + h) * height

    def __crop_image_to_bbox(self,image, X_meta_srs):
        x_min,  y_min, x_max, y_max = self.__get_bounds(image, X_meta_srs)
        im_crop = image.crop((x_min,  y_min, x_max, y_max))
        return im_crop

    
    def getCutImage(self,pathImage, arrayCOCO):
        image = Image.open(pathImage).convert("RGB")
        imgplot = self.__crop_image_to_bbox(image, arrayCOCO)
        return imgplot
    
#класс, который создает по папкам обрезанные фотки
class PrepareData(object):
    
    def createDirs(self):
        labelDir ={}
        os.mkdir(self.trainDir)

        for name in self.namesHands:
            newDir = os.path.join(self.trainDir, name)
            os.mkdir(newDir)
            labelDir[name] = newDir

        self.dirs = labelDir
    
    def __init__(self, namesHands, datasetDir):       
        
        #директория с обр фотками
        self.trainDir = os.path.join(datasetDir, 'cuts')
       
        #наименование рук для чтения json и папок с файлами
        self.namesHands = namesHands
        
        #папка с данными
        self.datasetDir = datasetDir
        
        #инициализация экз. утилиты для обрезки изображения 
        self.cutUtil = CutImageUtil()
        
        #создает подпапки с фото
        self.createDirs()
        
        #заполняем папки
        self.prepareData()
        

    #загрузка и обрезка фото 
    def prepareData(self):
        listTrainData = []

        for name in self.namesHands:
            with open(os.path.join(self.datasetDir, 'cut_' + name + '.json' ), "r") as my_file:
                capitals_json = my_file.read()

            file = json.loads(capitals_json)

            for i in file:
                if(file[i]['labels'][0] =='no_gesture'):
                    continue
                dic = {}
                dic['id'] = i
                dic['boxes'] = file[i]['bboxes'][0]
                dic['labels'] = file[i]['labels'][0]
                listTrainData.append(dic)
   
        for item in listTrainData:
            path = os.path.join(original_datasets_dir, "train_val_" + item['labels'] +'//' + item['id'] +'.jpg')
            cutedImage = self.cutUtil.getCutImage(path, item['boxes'])
            
            newDir =  os.path.join(self.trainDir, self.dirs[item['labels']]) 
            newDir =  os.path.join(newDir, item['id'] +'.jpg')
            
            cutedImage.save(newDir)

In [5]:
namesHands = ['call',
             'dislike',
             'like',
             'mute',
             'ok', 
             'rock',
             'stop']

original_datasets_dir = os.path.join(pathlib.Path().resolve(),"data")

modelDataa = PrepareData(namesHands, original_datasets_dir)